In [2]:
import pandas as pd
import numpy as np
import psycopg2
from sql import engine
import h3
import requests

pa_data = pd.read_csv('./data/Charging_PA.csv',sep=',')

/usr/local/Caskroom/miniconda/base/envs/sql-practice/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
charging_df = pd.DataFrame(pa_data)

In [4]:
charging_df.drop(['Start Time Zone','End Time Zone', 'Transaction Date (Pacific Time)',
                'GHG Savings (kg)', 'Gasoline Savings (gallons)','Address 1','City', 
                'State/Province', 'Postal Code', 'Country', 'County', 'System S/N', 
                'Model Number','Org Name','EVSE ID','Energy (kWh)','Plug Type','Currency'], inplace= True, axis= 1)

In [5]:
charging_df.columns = charging_df.columns.str.replace(' ', '_')
charging_df.columns = charging_df.columns.str.lower()
charging_df.columns = charging_df.columns.str.rstrip()

In [6]:
charging_df = charging_df.rename(columns={'total_duration_(hh:mm:ss)': 'total_duration', "charging_time_(hh:mm:ss)": "charging_time"})

In [7]:
# Make datetime columns only for 2019
date_cols = ["start_date", "end_date"]
duration_cols = ["total_duration", "charging_time"]

for col in date_cols:
     charging_df[date_cols] = charging_df[date_cols].apply(pd.to_datetime, errors = "coerce")


for col in duration_cols:
     charging_df[duration_cols] = charging_df[duration_cols].apply(pd.to_timedelta, errors="coerce")

In [8]:
charging_2020 = charging_df.query('start_date >= "2020-01-01" & start_date <= "2020-12-31"')
charging_2019 = charging_df.query('start_date >= "2019-01-01" & start_date <= "2019-12-31"')
charging_2018 = charging_df.query('start_date >= "2018-01-01" & start_date <= "2018-12-31"')
charging_2017 = charging_df.query('start_date >= "2017-01-01" & start_date <= "2017-12-31"')
charging_2016 = charging_df.query('start_date >= "2016-01-01" & start_date <= "2016-12-31"')
charging_2015 = charging_df.query('start_date >= "2015-01-01" & start_date <= "2015-12-31"')
charging_2014 = charging_df.query('start_date >= "2014-01-01" & start_date <= "2014-12-31"')
charging_2013 = charging_df.query('start_date >= "2013-01-01" & start_date <= "2013-12-31"')
charging_2012 = charging_df.query('start_date >= "2012-01-01" & start_date <= "2012-12-31"')
charging_2011 = charging_df.query('start_date >= "2011-01-01" & start_date <= "2011-12-31"')

In [9]:
charging_2019.head()

,station_name,mac_address,start_date,end_date,total_duration,charging_time,port_type,port_number,latitude,longitude,fee,ended_by,plug_in_event_id,driver_postal_code,user_id
191586,PALO ALTO CA / WEBSTER #3,0024:B100:0002:4277,2019-01-01 08:46:00,2019-01-01 10:58:00,0 days 02:11:14,0 days 02:06:39,Level 2,1,37.448273,-122.158234,2.86,Plug Out at Vehicle,1546361212,94601.0,492265.0
191587,PALO ALTO CA / BRYANT #6,0024:B100:0002:6B43,2019-01-01 09:18:00,2019-01-01 10:25:00,0 days 01:07:23,0 days 01:06:51,Level 2,1,37.446667,-122.162170,1.52,Plug Out at Vehicle,2951,94025.0,3174411.0
191588,PALO ALTO CA / BRYANT #1,000D:6F00:009D:9A1B,2019-01-01 09:19:00,2019-01-01 18:36:00,0 days 09:16:18,0 days 03:34:31,Level 2,2,37.446613,-122.162308,2.65,Plug Out at Vehicle,3524,94404.0,2358991.0
191589,PALO ALTO CA / BRYANT #5,0024:B100:0002:6D51,2019-01-01 09:46:00,2019-01-01 11:35:00,0 days 01:49:47,0 days 01:49:26,Level 2,1,37.446690,-122.162231,1.40,Plug Out at Vehicle,2792,94025.0,555879.0
191590,PALO ALTO CA / HIGH #4,0024:B100:0002:1327,2019-01-01 09:54:00,2019-01-01 11:25:00,0 days 01:31:12,0 days 01:30:53,Level 2,2,37.443138,-122.163239,2.08,Plug Out at Vehicle,2416,94305.0,3171851.0


In [10]:
charging_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47655 entries, 191586 to 239240
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   station_name        47655 non-null  object         
 1   mac_address         47655 non-null  object         
 2   start_date          47655 non-null  datetime64[ns] 
 3   end_date            47655 non-null  datetime64[ns] 
 4   total_duration      47655 non-null  timedelta64[ns]
 5   charging_time       47655 non-null  timedelta64[ns]
 6   port_type           47655 non-null  object         
 7   port_number         47655 non-null  int64          
 8   latitude            47655 non-null  float64        
 9   longitude           47655 non-null  float64        
 10  fee                 47655 non-null  float64        
 11  ended_by            47638 non-null  object         
 12  plug_in_event_id    47655 non-null  int64          
 13  driver_postal_code  46251

In [11]:
# replace NaT in end_date with start_date + total_duration
charging_2019['end_date'] = charging_2019['end_date'].fillna((charging_2019['start_date'] + charging_2019['total_duration'])).copy()

<ipython-input-11-704d1bc1865b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['end_date'] = charging_2019['end_date'].fillna((charging_2019['start_date'] + charging_2019['total_duration'])).copy()


In [12]:
# create column fee_bol (True if user paid, False if charging was free), drop original fee column
charging_2019['fee_bol'] = np.where(charging_2019['fee']== 0.0, False, True)
charging_2019.drop(['fee'],inplace= True, axis= 1)

<ipython-input-12-99e212b80dbc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['fee_bol'] = np.where(charging_2019['fee']== 0.0, False, True)
/usr/local/Caskroom/miniconda/base/envs/sql-practice/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
# create colum parking_time to display parking time (plugged in but not charging)
charging_2019['parking_time'] = charging_2019['total_duration'] - charging_2019['charging_time']

<ipython-input-13-e1a37cd582e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['parking_time'] = charging_2019['total_duration'] - charging_2019['charging_time']


Which year do we want to analyze?

unique identifier for each station plug = combination of plug ID and station name

In [14]:
# create column that displays weekday based on start_date
charging_2019['weekday'] = charging_2019['start_date'].dt.dayofweek

<ipython-input-14-202811ed20f3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['weekday'] = charging_2019['start_date'].dt.dayofweek


In [15]:
mymapp = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}

charging_2019['weekday_name'] = charging_2019.weekday.map(mymapp)

# charging_2019['weekday_name'] = charging_2019.applymap(lambda s: mymapp.get(s) if s in mymapp else s)

<ipython-input-15-f86a720fc056>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['weekday_name'] = charging_2019.weekday.map(mymapp)


In [16]:
charging_2019.head()

,station_name,mac_address,start_date,end_date,total_duration,charging_time,port_type,port_number,latitude,longitude,ended_by,plug_in_event_id,driver_postal_code,user_id,fee_bol,parking_time,weekday,weekday_name
191586,PALO ALTO CA / WEBSTER #3,0024:B100:0002:4277,2019-01-01 08:46:00,2019-01-01 10:58:00,0 days 02:11:14,0 days 02:06:39,Level 2,1,37.448273,-122.158234,Plug Out at Vehicle,1546361212,94601.0,492265.0,True,0 days 00:04:35,1,Tuesday
191587,PALO ALTO CA / BRYANT #6,0024:B100:0002:6B43,2019-01-01 09:18:00,2019-01-01 10:25:00,0 days 01:07:23,0 days 01:06:51,Level 2,1,37.446667,-122.162170,Plug Out at Vehicle,2951,94025.0,3174411.0,True,0 days 00:00:32,1,Tuesday
191588,PALO ALTO CA / BRYANT #1,000D:6F00:009D:9A1B,2019-01-01 09:19:00,2019-01-01 18:36:00,0 days 09:16:18,0 days 03:34:31,Level 2,2,37.446613,-122.162308,Plug Out at Vehicle,3524,94404.0,2358991.0,True,0 days 05:41:47,1,Tuesday
191589,PALO ALTO CA / BRYANT #5,0024:B100:0002:6D51,2019-01-01 09:46:00,2019-01-01 11:35:00,0 days 01:49:47,0 days 01:49:26,Level 2,1,37.446690,-122.162231,Plug Out at Vehicle,2792,94025.0,555879.0,True,0 days 00:00:21,1,Tuesday
191590,PALO ALTO CA / HIGH #4,0024:B100:0002:1327,2019-01-01 09:54:00,2019-01-01 11:25:00,0 days 01:31:12,0 days 01:30:53,Level 2,2,37.443138,-122.163239,Plug Out at Vehicle,2416,94305.0,3171851.0,True,0 days 00:00:19,1,Tuesday


In [17]:
# def make_weekday_names(row):
#     if row['weekday'] == 0:
#         return 'Monday'
#     if row['weekday'] == 1:
#         return 'Tuesday'
#     if row['weekday'] == 2:
#         return 'Wednesday'
#     if row['weekday'] == 3:
#         return 'Thursday'
#     if row['weekday'] == 4:
#         return 'Friday'
#     if row['weekday'] == 5:
#         return 'Saturday'
#     if row['weekday'] == 6:
#         return 'Sunday'

In [18]:
# charging_2019['weekday_name'] = charging_2019.apply(lambda row : make_weekday_names(row), axis=1)

In [19]:
def make_mac_same(row):
    return row['mac_address'].replace(':','')

charging_2019['mac_address'] = charging_2019.apply(lambda row : make_mac_same(row), axis=1)

<ipython-input-19-ed39c61e56b2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['mac_address'] = charging_2019.apply(lambda row : make_mac_same(row), axis=1)


In [20]:
def make_unique_id(row):
    return str(row['mac_address']) + '_' +  str(row['port_number'])

charging_2019['unique_plug_id'] = charging_2019.apply(lambda row : make_unique_id(row), axis=1)

<ipython-input-20-9e905a49289f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['unique_plug_id'] = charging_2019.apply(lambda row : make_unique_id(row), axis=1)


In [21]:
charging_2019['index1'] = charging_2019.index

<ipython-input-21-b41f60fb34ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['index1'] = charging_2019.index


In [22]:
# change order of columns, put unique_plug_id in front, drop MAC ID
charging_2019 = charging_2019[['station_name',
                            'unique_plug_id', 
                            'start_date',
                            'end_date',
                            'total_duration',
                            'charging_time',
                            'parking_time',
                            'port_type',
                            'port_number',
                            'latitude',
                            'longitude',
                            'ended_by', 
                            'plug_in_event_id',
                            'driver_postal_code', 
                            'user_id',
                            'fee_bol', 
                            'weekday',
                            'weekday_name']]

In [23]:
charging_2019 = charging_2019.sort_values(['unique_plug_id', 'start_date'])
charging_2019 = charging_2019.reset_index()
charging_2019.drop(['index'], inplace= True, axis= 1)
charging_2019['idle_time'] = None

In [24]:
max_index = int(len(charging_2019) -1)

for index, row in charging_2019.iterrows():
    if index >= max_index:
        charging_2019['idle_time'][index] = None
    elif index <= max_index:
        if charging_2019['unique_plug_id'][index+1] != charging_2019['unique_plug_id'][index]:
            charging_2019['idle_time'][index] = None
        elif charging_2019['unique_plug_id'][index+1] == charging_2019['unique_plug_id'][index]:
            charging_2019['idle_time'][index] = charging_2019['start_date'][index+1] - charging_2019['end_date'][index]

<ipython-input-24-f94ba14cd3ef>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['idle_time'][index] = charging_2019['start_date'][index+1] - charging_2019['end_date'][index]
<ipython-input-24-f94ba14cd3ef>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['idle_time'][index] = None
<ipython-input-24-f94ba14cd3ef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['idle_time'][index] = None


In [25]:
duration_cols = ["idle_time"]

for col in duration_cols:
     charging_2019[duration_cols] = charging_2019[duration_cols].apply(pd.to_timedelta, errors="coerce")

In [26]:
# bring idle_time to front
charging_2019 = charging_2019[['station_name',
                            'unique_plug_id', 
                            'start_date',
                            'end_date',
                            'total_duration',
                            'charging_time',
                            'parking_time',
                            'idle_time',
                            'port_type',
                            'port_number',
                            'latitude',
                            'longitude',
                            'ended_by', 
                            'plug_in_event_id',
                            'driver_postal_code', 
                            'user_id',
                            'fee_bol', 
                            'weekday',
                            'weekday_name']]

In [27]:
# make column whether zip is from PA (1) or not (0)
# pa_zip_codes = ['94020', '94301', '94306', '94022', '94302', '95033', '94024', '94303', '94028', '94304']
# def determine_pa_zip(row):
#     if str(row['driver_postal_code']) in pa_zip_codes:
#         return True
#     elif str(row['driver_postal_code']) not in pa_zip_codes:
#         return False
# charging_2019.apply(lambda row: determine_pa_zip(row), axis=1)

In [28]:
# make column whether zip is from PA (1) or not (0)
pa_zip_codes = [94020, 94301, 94306, 94022, 94302, 95033, 94024, 94303, 94028, 94304]

charging_2019['pa_zip_code'] = charging_2019['driver_postal_code'].isin(pa_zip_codes)

In [29]:
charging_2019.pa_zip_code.nunique()

2

In [30]:
# determine local & out-of-city users 
# first get rid of decimal in postal code
charging_2019['driver_postal_code'] = charging_2019['driver_postal_code'].fillna(0).astype(int)
charging_2019['driver_postal_code'].nunique() # 571 unique postal codes

571

In [31]:
def make_h3_code(row):
    return h3.geo_to_h3(
            lat=row['latitude'],
            lng=row['longitude'],
            resolution=9    # Average Hexagon Area (km2): 0.7373276
                            # Average Hexagon Edge Length (km)
                            # https://h3geo.org/docs/core-library/restable/
                            # 8 = 0.7373276 km^2
                            # 9	= 0.1053325 km^2
)
charging_2019['h3_code'] = charging_2019.apply(lambda row : make_h3_code(row), axis=1)
charging_2019.h3_code.nunique()

8

In [32]:
# table_name = 'charging_df_nd_2019'
# # Write records stored in a dataframe to SQL database
# if engine!=None:
#     try:
#         charging_2019.to_sql(name=table_name, # Name of SQL table
#                         con=engine, # Engine or connection
#                         if_exists='replace', # Drop the table before inserting new values 
#                         index=False, # Write DataFrame index as a column
#                         chunksize=5000, # Specify the number of rows in each batch to be written at a time
#                         method='multi') # Pass multiple values in a single INSERT clause
#         print(f"The {table_name} table was imported successfully.")
#     # Error handling
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None

### Potential Research Questions

Goal: Incentivize consumers in Germany (MUC and surroundings) to buy an EV by providing an optimal charging infrastructure (this means: there is (almost) always a charging station available at the right place (residential, leisure, work)

Identifying charging behavior patterns in relation to location specifics - Is Palo Alto a best practice example? If yes, what can other cities (e.g. MUC) learn from them in terms of charging infrastructure?

* Impact of COVID -- if time permits
* crisis 2013/2014 -- if time permits
* identify usage patterns according to location 
    * What are peak hours? Difference between weekdays / weekends?, per month? per year? 
    * peak times: How many PA users and how many from outside?
    * avg. charging time, avg. parking time, idle times
    * How many users are not from PA? What does MUC have to take into account in terms of "outside"/"shuttle" users?
    * Analyze recurring charging customers with zip codes not from PA? Which stations do they use? 
* What infrastructure strategy is PA using (e.g. how many new stations per year, where? (location characteristics); Is Palo Alto best practice? (define KPIs, what makes a best practice? Ideas: high idle times (even during peak hours or at least other free station nearby), high ratio of charging stations per EV, high coverage?)
* How does MUC compare?
* Which stations were added newly each year? compare with MUC

Potential categories: living, working, shopping, recreation

In [33]:
# h3 to geo returns a tuple, so we have to convert it into a list to access it right

h3_list = charging_2019.h3_code.unique()
geo_center_list = []

for i in h3_list:
    k = list(h3.h3_to_geo(i))
    geo_center_list.append(k)

geo_center_list_string = []

def make_string_geo():
    for i in geo_center_list:
        string = str(i[0]) + ',' + str(i[1])
        geo_center_list_string.append(string)
make_string_geo()


In [34]:
api_key= 'Z1XB-w87ksYmy5gJnj4gfLPJUJRkiyyPDLEtyjnMYy0'
# url_range = f'https://browse.search.hereapi.com/v1/browse?at={GEO}&in=circle:{GEO};r=700&limit=100&apiKey={api_key}'

title_list = []
cat_id_list = []
cat_def_list = []
cat_name_list = []
lat_list = []
lng_list = []
source_geo = []
h3_source_list = []


count = 0

for index, GEO in enumerate(geo_center_list_string):
    resp = requests.get(url = f'https://browse.search.hereapi.com/v1/browse?at={GEO}&in=circle:{GEO};r=200&limit=100&apiKey={api_key}')
    data= resp.json()

    for i in data['items']:
        if 'categories' not in i:
            continue
        else:
            title_list.append(str(i['title']))
            cat_id_list.append(str(i['categories'][0]['id']))
            cat_name_list.append(str(i['categories'][0]['name']))
            lat_list.append(str(i['position']['lat']))
            lng_list.append(str(i['position']['lng']))
            source_geo.append(GEO)
            h3_source_list.append(h3_list[index])
            count +=1
print('Places nearby: ' + str(count))

Places nearby: 609


In [35]:
dict = {'title': title_list, 'cat_id': cat_id_list, 'cat_name': cat_name_list, 'latitude': lat_list, 'longitude': lng_list, 'geo_source': source_geo, 'h3_source': h3_source_list} 
cat_df = pd.DataFrame(dict)

In [36]:
# searched for 1 instead of 100 because we had null-values in cat 500 ( there was a 550 at the beginning)

def make_cat_def(row):
    if row['cat_id'][:1] == '1':
        return 'Eat & Drink'
    elif row['cat_id'][:1] == '2':
        return 'Going Out-Entertainment'
    elif row['cat_id'][:1] == '3':
        return 'Sights and Museums'
    elif row['cat_id'][:1] == '4':
        return 'Transport'
    elif row['cat_id'][:1] == '5':
        return 'Accommodations'
    elif row['cat_id'][:1] == '6':
        return 'Shopping'
    elif row['cat_id'][:1] == '7':
        return 'Business and Services'
    elif row['cat_id'][:1] == '8':
        return 'Facilities'
    elif row['cat_id'][:1] == '9':
        return 'Areas and Buildings'

cat_df['cat_definition'] = cat_df.apply(lambda row : make_cat_def(row), axis=1)

In [37]:
cat_df = cat_df[[   'h3_source',
                    'cat_definition', 
                    'cat_name',
                    'title'
                ]]

In [38]:
cat_df.sort_values(['h3_source', 'cat_definition'])

,h3_source,cat_definition,cat_name,title
180,8928347432fffff,Accommodations,Hotel/Motel,Nobu Hotel Palo Alto
123,8928347432fffff,Business and Services,Real Estate Services,Kate Fomina - Princeton Real Estate
124,8928347432fffff,Business and Services,Finance and Insurance,Raymond James Financial Services
125,8928347432fffff,Business and Services,Management and Consulting Services,Silvertree Media
126,8928347432fffff,Business and Services,Business Services,Buzz All Stars
...,...,...,...,...
88,89283475d97ffff,Shopping,Hair & Beauty,Nails by Michelle
89,89283475d97ffff,Shopping,Hair Salon,Balayage Lynn
90,89283475d97ffff,Shopping,Hair Salon,Betty's Hair Studio
94,89283475d97ffff,Shopping,Hair & Beauty,Olivia & Lash


In [39]:
new_df_dict = { 
                'h3_code': h3_list, 
                'station_count': None, 
                'plug_count': None, 
                'Eat_and_Drink': None, 
                'Going_Out-Entertainment': None, 
                'Sights_and_Museums': None, 
                'Transport': None,
                'Accommodations': None, 
                'Shopping': None, 
                'Business_and_Services': None,
                'Facilities': None, 
                'Areas_and_Buildings': None}
master_compare_df = pd.DataFrame(new_df_dict)

In [40]:
charging_2019.head(1)

,station_name,unique_plug_id,start_date,end_date,total_duration,charging_time,parking_time,idle_time,port_type,port_number,...,longitude,ended_by,plug_in_event_id,driver_postal_code,user_id,fee_bol,weekday,weekday_name,pa_zip_code,h3_code
0,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-02-01 11:55:00,2019-02-01 12:41:00,0 days 00:46:11,0 days 00:44:25,0 days 00:01:46,10 days 00:23:00,Level 1,1,...,-122.162308,Customer,3604,95008,265179.0,True,4,Friday,False,89283475d97ffff


In [41]:
def count_station_in_h3(row):
    return charging_2019[charging_2019.h3_code == row['h3_code']].nunique()['station_name']  
def count_plug_id_in_h3(row):
    return charging_2019[charging_2019.h3_code == row['h3_code']].nunique()['unique_plug_id']
def count_100_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Eat & Drink')].count()
def count_200_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Going Out-Entertainment')].count()
def count_300_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Sights and Museums')].count()
def count_400_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Transport')].count()
def count_500_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Accommodations')].count()
def count_600_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Shopping')].count()
def count_700_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Business and Services')].count()
def count_800_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Facilities')].count()
def count_900_in_h3(row):
    return cat_df[(cat_df.h3_source == row['h3_code']) & (cat_df.cat_definition == 'Areas and Buildings')].count()
def total_poi(row):
    return row['Eat_and_Drink']+row['Going_Out-Entertainment']+row['Sights_and_Museums']+row['Transport']+row['Accommodations']+row['Shopping']+row['Business_and_Services']+row['Facilities']+row['Areas_and_Buildings']
def count_inner_user(row):
    return charging_2019[(charging_2019.h3_code == row['h3_code'])].sum()['pa_zip_code']
def count_all_user(row):
    return charging_2019[(charging_2019.h3_code == row['h3_code'])].count()['pa_zip_code']
def calc_outer_user(row):
    return row.all_user - row.inner_user




In [42]:
master_compare_df['station_count'] = master_compare_df.apply(lambda row : count_station_in_h3(row), axis=1)
master_compare_df['plug_count'] = master_compare_df.apply(lambda row : count_plug_id_in_h3(row), axis=1)
master_compare_df['Eat_and_Drink'] = master_compare_df.apply(lambda row : count_100_in_h3(row), axis=1)
master_compare_df['Going_Out-Entertainment'] = master_compare_df.apply(lambda row : count_200_in_h3(row), axis=1)
master_compare_df['Sights_and_Museums'] = master_compare_df.apply(lambda row : count_300_in_h3(row), axis=1)
master_compare_df['Transport'] = master_compare_df.apply(lambda row : count_400_in_h3(row), axis=1)
master_compare_df['Accommodations'] = master_compare_df.apply(lambda row : count_500_in_h3(row), axis=1)
master_compare_df['Shopping'] = master_compare_df.apply(lambda row : count_600_in_h3(row), axis=1)
master_compare_df['Business_and_Services'] = master_compare_df.apply(lambda row : count_700_in_h3(row), axis=1)
master_compare_df['Facilities'] = master_compare_df.apply(lambda row : count_800_in_h3(row), axis=1)
master_compare_df['Areas_and_Buildings'] = master_compare_df.apply(lambda row : count_900_in_h3(row), axis=1)
master_compare_df['total_poi'] = master_compare_df.apply(lambda row : total_poi(row), axis=1)
master_compare_df['inner_user'] = master_compare_df.apply(lambda row : count_inner_user(row), axis=1)
master_compare_df['all_user'] = master_compare_df.apply(lambda row : count_all_user(row), axis=1)
master_compare_df['outer_user'] = master_compare_df.apply(lambda row : calc_outer_user(row), axis=1)


In [43]:
master_compare_df.head(10)

,h3_code,station_count,plug_count,Eat_and_Drink,Going_Out-Entertainment,Sights_and_Museums,Transport,Accommodations,Shopping,Business_and_Services,Facilities,Areas_and_Buildings,total_poi,inner_user,all_user,outer_user
0,89283475d97ffff,6,12,16,2,1,0,0,27,41,12,0,99,1479,7654,6175
1,89283475ca3ffff,3,5,0,0,1,0,1,1,2,3,0,8,1239,2389,1150
2,89283475527ffff,6,8,1,0,1,2,1,1,2,4,0,12,1507,3714,2207
3,8928347432fffff,2,4,12,0,2,0,1,9,55,19,0,98,1314,5157,3843
4,89283475d83ffff,3,6,1,0,0,0,0,1,68,30,0,100,1485,6536,5051
5,89283474e5bffff,4,8,16,1,1,2,2,17,54,6,0,99,2010,7634,5624
6,892834743c3ffff,4,10,6,1,2,1,0,10,47,29,0,96,1408,5172,3764
7,892834743d3ffff,5,12,9,0,0,2,1,18,54,13,0,97,2438,9399,6961


In [44]:
cat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   h3_source       609 non-null    object
 1   cat_definition  609 non-null    object
 2   cat_name        609 non-null    object
 3   title           609 non-null    object
dtypes: object(4)
memory usage: 19.2+ KB


In [45]:
# determine local & out-of-city users
# first get rid of decimal in postal code
# charging_2019['driver_postal_code'] = charging_2019['driver_postal_code'].fillna(0).astype(int)
# charging_2019['driver_postal_code'].nunique() # 571 unique postal codes
# charging_2019.groupby('weekday').count()['driver_postal_code'].plot(kind='bar')
# charging_2019.groupby('weekday').count()['driver_postal_code'].plot(kind='bar')
# during the week we have a higher count of zip_codes --> lots of commuters to PA

In [46]:
# plot PA vs. non-PA zips per weekday
# charging_2019.groupby(['h3_code','weekday']).sum()['pa_zip_code'].plot(kind='bar')

In [47]:
# charging_2019.groupby(['h3_code', 'weekday'])['charging_time'].mean()

In [48]:
# charging_2019.groupby(['h3_code', 'weekday'])['idle_time'].mean()

In [49]:
# charging_2019.groupby(['h3_code', 'weekday'])['parking_time'].mean()

In [50]:
charging_2019['total_duration_min']= round(charging_2019['total_duration'] / np.timedelta64(1, 'm'),0)
charging_2019['total_idle_time_min']= round(charging_2019['idle_time'] / np.timedelta64(1, 'm'),0)
charging_2019['total_parking_min']= round(charging_2019['parking_time'] / np.timedelta64(1, 'm'),0)
charging_2019['total_charging_min']= round(charging_2019['charging_time'] / np.timedelta64(1, 'm'),0)


In [51]:
charging_2019.head(5)

,station_name,unique_plug_id,start_date,end_date,total_duration,charging_time,parking_time,idle_time,port_type,port_number,...,user_id,fee_bol,weekday,weekday_name,pa_zip_code,h3_code,total_duration_min,total_idle_time_min,total_parking_min,total_charging_min
0,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-02-01 11:55:00,2019-02-01 12:41:00,0 days 00:46:11,0 days 00:44:25,0 days 00:01:46,10 days 00:23:00,Level 1,1,...,265179.0,True,4,Friday,False,89283475d97ffff,46.0,14423.0,2.0,44.0
1,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-02-11 13:04:00,2019-02-11 16:03:00,0 days 02:58:07,0 days 02:53:48,0 days 00:04:19,21 days 07:19:00,Level 1,1,...,208307.0,True,0,Monday,True,89283475d97ffff,178.0,30679.0,4.0,174.0
2,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-03-04 23:22:00,2019-03-04 23:24:00,0 days 00:02:14,0 days 00:01:19,0 days 00:00:55,6 days 11:17:00,Level 1,1,...,705633.0,False,0,Monday,False,89283475d97ffff,2.0,9317.0,1.0,1.0
3,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-03-11 10:41:00,2019-03-11 12:54:00,0 days 02:13:45,0 days 02:09:49,0 days 00:03:56,241 days 22:59:00,Level 1,1,...,2589921.0,True,0,Monday,False,89283475d97ffff,134.0,348419.0,4.0,130.0
4,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-11-08 11:53:00,2019-11-08 12:59:00,0 days 01:06:11,0 days 01:04:05,0 days 00:02:06,NaT,Level 1,1,...,208307,True,4,Friday,True,89283475d97ffff,66.0,NaN,2.0,64.0


In [52]:
charging_2019.to_csv('charging_2019_df(small).csv', encoding='utf-8', index=False)
master_compare_df.to_csv('master_compare_df(small).csv', encoding='utf-8', index=False)

In [53]:
charging_2019.head()

,station_name,unique_plug_id,start_date,end_date,total_duration,charging_time,parking_time,idle_time,port_type,port_number,...,user_id,fee_bol,weekday,weekday_name,pa_zip_code,h3_code,total_duration_min,total_idle_time_min,total_parking_min,total_charging_min
0,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-02-01 11:55:00,2019-02-01 12:41:00,0 days 00:46:11,0 days 00:44:25,0 days 00:01:46,10 days 00:23:00,Level 1,1,...,265179.0,True,4,Friday,False,89283475d97ffff,46.0,14423.0,2.0,44.0
1,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-02-11 13:04:00,2019-02-11 16:03:00,0 days 02:58:07,0 days 02:53:48,0 days 00:04:19,21 days 07:19:00,Level 1,1,...,208307.0,True,0,Monday,True,89283475d97ffff,178.0,30679.0,4.0,174.0
2,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-03-04 23:22:00,2019-03-04 23:24:00,0 days 00:02:14,0 days 00:01:19,0 days 00:00:55,6 days 11:17:00,Level 1,1,...,705633.0,False,0,Monday,False,89283475d97ffff,2.0,9317.0,1.0,1.0
3,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-03-11 10:41:00,2019-03-11 12:54:00,0 days 02:13:45,0 days 02:09:49,0 days 00:03:56,241 days 22:59:00,Level 1,1,...,2589921.0,True,0,Monday,False,89283475d97ffff,134.0,348419.0,4.0,130.0
4,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-11-08 11:53:00,2019-11-08 12:59:00,0 days 01:06:11,0 days 01:04:05,0 days 00:02:06,NaT,Level 1,1,...,208307,True,4,Friday,True,89283475d97ffff,66.0,NaN,2.0,64.0


In [55]:
charging_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47655 entries, 0 to 47654
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   station_name         47655 non-null  object         
 1   unique_plug_id       47655 non-null  object         
 2   start_date           47655 non-null  datetime64[ns] 
 3   end_date             47655 non-null  datetime64[ns] 
 4   total_duration       47655 non-null  timedelta64[ns]
 5   charging_time        47655 non-null  timedelta64[ns]
 6   parking_time         47655 non-null  timedelta64[ns]
 7   idle_time            47590 non-null  timedelta64[ns]
 8   port_type            47655 non-null  object         
 9   port_number          47655 non-null  int64          
 10  latitude             47655 non-null  float64        
 11  longitude            47655 non-null  float64        
 12  ended_by             47638 non-null  object         
 13  plug_in_event_id

In [58]:
charging_2019['start_hour'] = charging_2019.start_date.dt.hour

In [59]:
charging_2019.head()

,station_name,unique_plug_id,start_date,end_date,total_duration,charging_time,parking_time,idle_time,port_type,port_number,...,fee_bol,weekday,weekday_name,pa_zip_code,h3_code,total_duration_min,total_idle_time_min,total_parking_min,total_charging_min,start_hour
0,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-02-01 11:55:00,2019-02-01 12:41:00,0 days 00:46:11,0 days 00:44:25,0 days 00:01:46,10 days 00:23:00,Level 1,1,...,True,4,Friday,False,89283475d97ffff,46.0,14423.0,2.0,44.0,11
1,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-02-11 13:04:00,2019-02-11 16:03:00,0 days 02:58:07,0 days 02:53:48,0 days 00:04:19,21 days 07:19:00,Level 1,1,...,True,0,Monday,True,89283475d97ffff,178.0,30679.0,4.0,174.0,13
2,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-03-04 23:22:00,2019-03-04 23:24:00,0 days 00:02:14,0 days 00:01:19,0 days 00:00:55,6 days 11:17:00,Level 1,1,...,False,0,Monday,False,89283475d97ffff,2.0,9317.0,1.0,1.0,23
3,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-03-11 10:41:00,2019-03-11 12:54:00,0 days 02:13:45,0 days 02:09:49,0 days 00:03:56,241 days 22:59:00,Level 1,1,...,True,0,Monday,False,89283475d97ffff,134.0,348419.0,4.0,130.0,10
4,PALO ALTO CA / BRYANT #1,000D6F00009D9A1B_1,2019-11-08 11:53:00,2019-11-08 12:59:00,0 days 01:06:11,0 days 01:04:05,0 days 00:02:06,NaT,Level 1,1,...,True,4,Friday,True,89283475d97ffff,66.0,NaN,2.0,64.0,11


In [61]:
table_name = 'charging_2019_nd'
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        charging_2019.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

/usr/local/Caskroom/miniconda/base/envs/sql-practice/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  sql.to_sql(


The charging_2019_nd table was imported successfully.


In [62]:
table_name = 'h3_categories'
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        master_compare_df.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The h3_categories table was imported successfully.


In [63]:
charging_2019.corr()

,port_number,latitude,longitude,plug_in_event_id,driver_postal_code,fee_bol,weekday,pa_zip_code,total_duration_min,total_idle_time_min,total_parking_min,total_charging_min,start_hour
port_number,1.000000,0.234524,-0.237324,-0.016004,0.004228,0.004377,-0.011245,-0.004719,0.032286,-0.019474,0.015892,0.032341,0.000024
latitude,0.234524,1.000000,-0.772523,-0.015778,-0.024486,0.012391,-0.035014,-0.054215,0.052197,-0.012472,0.006308,0.071379,0.063609
longitude,-0.237324,-0.772523,1.000000,0.015163,0.019685,-0.015863,0.048238,0.123162,-0.010026,0.017766,0.013011,-0.027804,-0.023796
plug_in_event_id,-0.016004,-0.015778,0.015163,1.000000,0.003339,-0.012157,-0.017352,-0.014543,0.035427,0.002803,0.021002,0.031948,0.001013
driver_postal_code,0.004228,-0.024486,0.019685,0.003339,1.000000,0.021890,0.000218,0.111828,-0.051981,0.002347,-0.031143,-0.046506,-0.031105
fee_bol,0.004377,0.012391,-0.015863,-0.012157,0.021890,1.000000,-0.000035,0.008650,0.075983,0.003114,0.002070,0.110769,-0.018838
weekday,-0.011245,-0.035014,0.048238,-0.017352,0.000218,-0.000035,1.000000,0.085362,-0.087872,0.004764,-0.018906,-0.111984,0.068880
pa_zip_code,-0.004719,-0.054215,0.123162,-0.014543,0.111828,0.008650,0.085362,1.000000,-0.034055,0.004112,-0.007720,-0.043059,0.078189
total_duration_min,0.032286,0.052197,-0.010026,0.035427,-0.051981,0.075983,-0.087872,-0.034055,1.000000,0.003939,0.743844,0.751616,-0.073311
total_idle_time_min,-0.019474,-0.012472,0.017766,0.002803,0.002347,0.003114,0.004764,0.004112,0.003939,1.000000,-0.000020,0.005916,0.047928


In [74]:
def total_duration_mean(row):
    return charging_2019[(charging_2019.h3_code == row['h3_code'])].mean()['total_duration_min']
def total_charging_mean(row):
    return charging_2019[(charging_2019.h3_code == row['h3_code'])].mean()['total_charging_min']
def total_parking_mean(row):
    return charging_2019[(charging_2019.h3_code == row['h3_code'])].mean()['total_parking_min']
def total_idle_mean(row):
    return charging_2019[(charging_2019.h3_code == row['h3_code'])].median()['total_idle_time_min']            


master_compare_df['total_duration_min'] = master_compare_df.apply(lambda row : total_duration_mean(row), axis=1)
master_compare_df['total_charging_min'] = master_compare_df.apply(lambda row : total_charging_mean(row), axis=1)
master_compare_df['total_parking_min'] = master_compare_df.apply(lambda row : total_parking_mean(row), axis=1)
master_compare_df['total_idle_time_min'] = master_compare_df.apply(lambda row : total_idle_mean(row), axis=1)



<ipython-input-74-05b44db941c8>:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  return charging_2019[(charging_2019.h3_code == row['h3_code'])].mean()['total_duration_min']
<ipython-input-74-05b44db941c8>:4: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  return charging_2019[(charging_2019.h3_code == row['h3_code'])].mean()['total_charging_min']
<ipython-input-74-05b44db941c8>:6: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  return charging_2019[(charging_2019.h3_code == row['h3_code'])].mean()['total_parking_min']
<ipython-input-74-05b44db941c8>:8: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.


In [75]:
master_compare_df.head()

,h3_code,station_count,plug_count,Eat_and_Drink,Going_Out-Entertainment,Sights_and_Museums,Transport,Accommodations,Shopping,Business_and_Services,Facilities,Areas_and_Buildings,total_poi,inner_user,all_user,outer_user,total_duration_min,total_charging_min,total_parking_min,total_idle_time_min
0,89283475d97ffff,6,12,16,2,1,0,0,27,41,12,0,99,1479,7654,6175,142.006663,123.399791,18.483015,175.0
1,89283475ca3ffff,3,5,0,0,1,0,1,1,2,3,0,8,1239,2389,1150,174.983675,136.486396,38.345751,188.0
2,89283475527ffff,6,8,1,0,1,2,1,1,2,4,0,12,1507,3714,2207,120.957997,105.123317,15.690630,89.0
3,8928347432fffff,2,4,12,0,2,0,1,9,55,19,0,98,1314,5157,3843,123.537522,109.386077,14.033547,43.0
4,89283475d83ffff,3,6,1,0,0,0,0,1,68,30,0,100,1485,6536,5051,146.587209,129.812271,16.649174,83.0


In [78]:
correlation = master_compare_df.corr()

In [82]:
import sea

ModuleNotFoundError: No module named 'seaborn'